# Reddit Query Tool

## What is it?
This tool lets you query information from the [Reddit Pushshift API](https://github.com/pushshift/api). With this tool you can:

- Explore how often Reddit users have posted about a given topic
- Find sample posts about a given topic
- Get top-level metrics about Reddit posts

## How can I use it?
Simply add in your query into the below fields. All fields are optional. Please ensure that all fields, including blank fields, are wrapped in single quotes ('').

Once you have added in all of your query info hit "Cell" -> "Run all" (https://screenshot.googleplex.com/mPuJGiqfKMK.png).

## More info
For questions, comments, feature requests, more information, or a friendly chat reach out to zmpollack@.

In [9]:
# Search by specific post IDs as a comma separated list. 
# Example: ids = 'abcd, wxyz, qwerty'
ids = ''

# Keyword search terms as a comma separated list. Muli-word phrases should be wrapped in double quotes. 
# Example: q = 'android, "find my device", battery'
q = ''

# Exlude a keyword in the search. Formatted as a comma separated list. Muli-word phrases should be wrapped in double quotes. 
# Example: q_not: 'robots, "book recommendations", ios'
q_not = ''

# Search just post titles by keyword. Formatted as a comma separated list. Muli-word phrases should be wrapped in double quotes.
# Example: title = 'android, "find my device", battery'
title = ''

# Exclude a keyword in post titles. Formatted as a comma separated list. Muli-word phrases should be wrapped in double quotes.
# Example: titlenot = 'robots, "book recommendations", ios'
titlenot = ''

# Number of posts to be returned. Can be between 0 - 500. Default is 25.
# Exmple: size = '10'
size = ''

# Search by post author. Only one author can be specified. 
# Example: author = 'sundar'
author = ''

# Limit the search to a specfic subreddit. Only one subreddit can be specified. No need to include "r/".
# Example: subreddit = 'google'
subreddit = ''

# Search for submissions posted after a given date. Format must be mm/dd/yyyy.
# Example: after = '09/27/19'
after = ''

# Search for submissions posted before a given date. Format must be mm/dd/yyyy.
# Example: before = '12/31/2019'
before = ''

# Restrict the search to posts based on post score as expressed using '>' or '<'.
# Example: score = '>100'
score = ''

# Limit search to posts with a given number of comments as expressed using '>' or '<'.
# Example: num_comments = '>50'
num_comments = ''

In [10]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

import requests
import json
import time
import os
import urllib

def convertTime():
    global after
    global before
    pattern = '%m/%d/%Y'
    os.environ['TZ']='UTC'
    
    if after == '':
        pass
    else:
        after = str(int(time.mktime(time.strptime(after, pattern))))
    if before == '':
        pass
    else:
        before = str(int(time.mktime(time.strptime(before, pattern))))
   
def createRequest():
    global q
    global q_not
    
    if q == '':
        pass
    else:
        q = urllib.parse.quote_plus(q)
        
    if q_not == '':
        pass
    else:
        q_not = urllib.parse.quote_plus(q_not)
    
    request = 'https://api.pushshift.io/reddit/search/submission/?'
    request += 'size=' + size + '&'
    request += "ids=" + ids + "&"
    request += "q=" + q + "&"
    request += "q:not=" + q_not + "&"
    request += "title=" + title + "&"
    request += "title:not=" + titlenot + "&"
    request += "author=" + author + "&"
    request += "subreddit=" + subreddit + "&"
    request += "after=" + after + "&"
    request += "before=" + before + "&"
    request += "score=" + score + "&"
    request += "num_comments=" + num_comments + "&"
    request += 'metadata=true' 
    
    return request
    
def getResponse(request):
    r = requests.get(request)
    result = json.loads(r.content)
    return result
    
def formatResponse(args):
    
    metadata = args['metadata']
    total_results = metadata['total_results']
    results_returned = metadata['results_returned']
    
    metadata_str = """
    **Info** \n \n
    Total submissions: %s \n
    Posts returned: %s \n
    *************************** \n
    """ %(total_results, results_returned)
    
    data = args['data']

    
    i = 0 
    data_str = '**Posts** \n \n'
    
    while i < results_returned:
        author = data[i]['author']
        full_link = data[i]['full_link']
        num_comments = data[i]['num_comments']
        subreddit = data[i]['subreddit']
        title = data[i]['title']
        
        data_str += """
        Title : %s \n
        Author: %s \n
        Subreddit: %s \n
        Link: %s \n
        Number of comments: %s \n
        *************************** \n
        """ %(title, author, subreddit, full_link, num_comments)
        
        i += 1
    
    full_str = str(metadata_str) + str(data_str)
    
    return(full_str)

convertTime()

request = createRequest()

response = getResponse(request)
    
print(formatResponse(response))


    **Info** 
 

    Total submissions: 6 

    Posts returned: 2 

    *************************** 

    **Posts** 
 

        Title : FYI: Find My Device doesn't need to be on in 'Device admin apps' to remotely wipe or lock your phone on Android 10 

        Author: RemarkableWork 

        Subreddit: Android 

        Link: https://www.reddit.com/r/Android/comments/eafdzj/fyi_find_my_device_doesnt_need_to_be_on_in_device/ 

        Number of comments: 411 

        *************************** 

        
        Title : "Find my device". Why is it so bad at location finding, when Google Maps Timeline perfectly tracks me daily? 

        Author: MotherPotential 

        Subreddit: Android 

        Link: https://www.reddit.com/r/Android/comments/bfbw55/find_my_device_why_is_it_so_bad_at_location/ 

        Number of comments: 18 

        *************************** 

        
